In [1]:
# check if pyspark works
import pyspark
print(pyspark.__version__)

3.5.1


In [2]:
# Load dataset using pyspark and clean it
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import isnan, when, count, col

# Initialize Spark session
spark = SparkSession.builder.appName("CarPricePrediction").getOrCreate()

# Load the dataset
data_path = r"./dataset/used_cars_data.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Show the first few rows of the dataframe
df.show(5)


+-----------------+------------+----+----------+----------+---------------+-----+--------+-----------------+--------------------+------------+----------+--------------------+----------------+-------------------+-----------+--------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+-------+--------------------+----------+--------------------+-----+------------+------+------+---------+--------+--------+-----------+-------------+----------+---------+--------------------+--------------------+----------+---------------+-------+---------------+-----------+------------------+-------+-------+--------------+-------------+------+-------------------+-----------+--------------------+------------+--------------------+------+------------+-----------------------+------------+--------------------+---------+-------+----+
|              vin|back_legroom| bed|bed_height|bed_length|      body_type|cabin|    city|city_fuel_economy|combine_fuel_

## Data Exploration and Preprocessing

Explore the data and preprocess it as needed.

In [3]:
def count_nulls(df):
    null_counts = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
    return null_counts


# Display the schema of the dataset
#df.printSchema()

# Display summary statistics
#df.describe().show()

# Drop unnecessary columns

# TODO drop the rows 
df = df.drop('bed', 'bed_height', 'bed_length', 'cabin', 'description', 'fleet', 'franchise_dealer', 'city_fuel_economy', 'highway_fuel_economy', 'salvage', 'savings_amount', 'franchise_make', 'combine_fuel_economy', 'daysonmarket', 'dealer_zip', 'is_certified', 'is_cpo', 'is_oemcpo', 'latitude', 'listing_color', 'listing_id', 'longitude', 'main_picture_url', 'owner_count', 'theft_title', 'torque', 'transmission_display', 'trimId', 'trim_name', 'vehicle_damage_category', 'wheel_system_display')

# Show rows number
#print(df.count())
# Display dataset description
#df.show()



In [4]:
# Assume null as false: 'frame_damaged', 'has_accidents', 'isCab', 'major_options' since usually if not indicates not exist
#count_nulls(df).show()
df = df.fillna({'frame_damaged': False, 'has_accidents': False, 'isCab': False, 'major_options': False})

#count_nulls(df).show()

In [5]:
# Handle missing values if any (example: drop rows with missing values)
df = df.dropna()

#print(df.count())
#count_nulls(df).show()

In [6]:
# Salva il dataset aggiornato
#df.write.csv('dataset_principale_aggiornato.csv', header=True)
df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mydata.csv")

In [9]:
from pyspark.sql.functions import col, coalesce, first
#df.coalesce(1).write.csv("./mydf")

#df.repartition(1).write.csv("./mydf")
df.repartition(1).write.option("header", True).csv("project-1/results")

In [ ]:
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("mydata.csv")

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, first

# Inizializza la sessione Spark
spark = SparkSession.builder.appName("FillMissingFuelEconomy").getOrCreate()

# Carica i dataset
dataset_principale = df
dataset_secondario = spark.read.csv('./dataset/vehicles.csv', header=True, inferSchema=True)

# Identifica i record con valori mancanti
missing_values_df = dataset_principale.filter(
    (col('highway_fuel_economy').isNull()) |
    (col('city_fuel_economy').isNull())
)

print(missing_values_df.count())

# Unione dinamica per riempire i valori mancanti
updated_df = df.alias("main").join(
    dataset_secondario.alias("sec"),
    (col("main.make_name") == col("sec.make")) &
    (col("main.model_name") == col("sec.model")) &
    (col("main.year") == col("sec.year")),
    how='left'
).select(
    "main.*",
    col("sec.highway08").alias("highway_fuel_economy"),
    col("sec.city08").alias("city_fuel_economy"),
    #"sec.model"  # Include the model from secondary dataset for grouping
)

updated_df.describe().show()



# Ferma la sessione Spark
#spark.stop()


234199


In [26]:
updated_df.show()
print(updated_df.count())

+-----------------+------------+---------------+--------+----------------+-------------------+-----------+--------------------+-------------+-------------+----------------+---------+-------------+-------+----------+--------------------+-----+------+--------+-----------+--------------------+----------+---------------+-------+------------------+------------------+-------+-------------+------+-------------------+------------+------------+---------+-------+----+--------------------+-----------------+
|              vin|back_legroom|      body_type|    city|engine_cylinders|engine_displacement|engine_type|      exterior_color|frame_damaged|front_legroom|fuel_tank_volume|fuel_type|has_accidents| height|horsepower|      interior_color|isCab|is_new|  length|listed_date|       major_options| make_name|maximum_seating|mileage|        model_name|             power|  price|seller_rating| sp_id|            sp_name|transmission|wheel_system|wheelbase|  width|year|highway_fuel_economy|city_fuel_econo

In [ ]:
df = df.dropna()
count_nulls(df).show()

In [ ]:
# Rimuovi i duplicati mantenendo solo il primo valore
final_df = updated_df.groupBy("make_name", "model_name", "year").agg(
    first("updated_highway_fuel_economy").alias("highway_fuel_economy"),
    first("updated_city_fuel_economy").alias("city_fuel_economy"),
)

final_df.describe().show()
# Unisci i record aggiornati con il dataset principale mantenendo i valori originali
final_df = dataset_principale.alias("original").join(
    final_df.alias("updated"),
    on=['make_name', 'model_name', 'year'],
    how='left'
).select(
    col("original.*"),
    coalesce(col("updated.highway_fuel_economy"), col("original.highway_fuel_economy")).alias("highway_fuel_economy"),
    coalesce(col("updated.city_fuel_economy"), col("original.city_fuel_economy")).alias("city_fuel_economy"),
)

# Salva il dataset aggiornato
final_df.write.csv('dataset_principale_aggiornato.csv', header=True)

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Carica i dataset
dataset_secondario = spark.read.csv('./dataset/vehicles.csv', header=True, inferSchema=True)

# Identifica i record con valori mancanti
missing_values_df = df.filter(
    (col('highway_fuel_economy').isNull()) |
    (col('city_fuel_economy').isNull())
)

# Aggiungi valori dal dataset secondario
updated_df = missing_values_df.alias("main").join(
    dataset_secondario.alias("sec"),
    (col("main.make_name") == col("sec.make")) &
    (col("main.model_name") == col("sec.model")) &
    (col("main.year") == col("sec.year")),
    how='left'
).select(
    "main.*",
    F.coalesce("highway_fuel_economy", "sec.highway08"),
    F.coalesce("city_fuel_economy", "sec.city08"),
)


updated_df.describe(80).show()
count_nulls(updated_df).show()




+-------+--------------------+------------------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+-----------------------------------------+-----------------------------------+
|summary|                 vin|      back_legroom|         body_type|               city|   city_fuel_econo

In [ ]:
# Unisci i record aggiornati con il dataset principale
final_df = df.alias("original").join(
    updated_df.alias("updated"),
    on=['make_name', 'model_name', 'year'],
    how='left'
).select(
    "original.*",
    F.coalesce("updated.highway_fuel_economy", "original.highway_fuel_economy").alias("highway_fuel_economy"),
    F.coalesce("updated.city_fuel_economy", "original.city_fuel_economy").alias("city_fuel_economy"),
    F.coalesce("updated.combine_fuel_economy", "original.combine_fuel_economy").alias("combine_fuel_economy")
)
# Salva il dataset aggiornato
final_df.write.csv('dataset_principale_aggiornato.csv', header=True)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `updated`.`combine_fuel_economy` cannot be resolved. Did you mean one of the following? [`updated`.`city_fuel_economy`, `updated`.`highway_fuel_economy`, `updated`.`back_legroom`, `original`.`city_fuel_economy`, `updated`.`front_legroom`].;
'Project [make_name#69422, model_name#69425, year#69445, vin#69380, back_legroom#69381, body_type#69385, city#69387, city_fuel_economy#69388, engine_cylinders#69393, engine_displacement#69394, engine_type#69395, exterior_color#69396, frame_damaged#72705, front_legroom#69401, fuel_tank_volume#69402, fuel_type#69403, has_accidents#72706, height#69405, highway_fuel_economy#69406, horsepower#69407, interior_color#69408, isCab#72707, is_new#69412, length#69415, ... 18 more fields]
+- Project [make_name#69422, model_name#69425, year#69445, vin#69380, back_legroom#69381, body_type#69385, city#69387, city_fuel_economy#69388, engine_cylinders#69393, engine_displac

In [ ]:
# drop also daysonmarket and dealer_zip not related to our study ... more general
# replace fro, datasets: highway_fuel_economy and city_fuel_economy eventually combine_fuel_economy
# source: https://www.fueleconomy.gov/feg/download.shtml
#Assume null as false: 'frame_damaged' and 'has_accidents', isCab, major_options
# Duplicates: sp_id and sp_name (name and model id it's same data...different version maybe int it's better but not understandable)
# Display the number of missing values in each column
count_nulls(df).show()

# Handle missing values if any (example: drop rows with missing values)
df = df.dropna()

print(df.count())
count_nulls(df).show()

+---+------------+---------+----+-----------------+----------------+-------------------+-----------+--------------+-------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------+-----+------+------+-----------+-------------+---------+---------------+-------+----------+-----+-----+-------+--------------+-------------+-----+-------+------------+------------+---------+-----+----+
|vin|back_legroom|body_type|city|city_fuel_economy|engine_cylinders|engine_displacement|engine_type|exterior_color|frame_damaged|front_legroom|fuel_tank_volume|fuel_type|has_accidents|height|highway_fuel_economy|horsepower|interior_color|isCab|is_new|length|listed_date|major_options|make_name|maximum_seating|mileage|model_name|power|price|salvage|savings_amount|seller_rating|sp_id|sp_name|transmission|wheel_system|wheelbase|width|year|
+---+------------+---------+----+-----------------+----------------+-------------------+-----------+--------------+-----